## Particle Swarm Optimization

### Particle Class and PSO implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Particle:
    def __init__(self, fitness, dim, minx, maxx, seed):
        np.random.seed(seed)
        self.pos = (maxx - minx) *np.random.rand(dim) + minx
        self.vel = (maxx - minx) *np.random.rand(dim) + minx
        self.fitness = fitness(self.pos)
        self.best_pos = np.copy(self.pos)
        self.best_fitness = self.fitness

def pso(fitness, max_iter, n, dim, minx, maxx, w, c1, c2):
    swarm = [Particle(fitness, dim, minx, maxx, i) for i in range(n)]
    best_particle = min(swarm,key=lambda x: x.fitness)
    swarm_best_pos = np.copy(best_particle.pos)
    swarm_best_fitness  = best_particle.fitness
    swarm_best_pos_track = [swarm_best_pos]
    for iter in range(max_iter):
        if iter % 10 == 0:
            print(f"{iter} > best fitness = {swarm_best_fitness:.4e} Best position: {swarm_best_pos}")

        for particle in swarm:
            r1,r2 = np.random.rand(2)
            particle.vel = (
                (w * particle.vel) +
                (c1 * r1 * (particle.best_pos - particle.pos)) +
                (c2 * r2 * (swarm_best_pos - particle.pos))
            )
            particle.vel = np.clip(particle.vel,minx,maxx)
            particle.pos += particle.vel
            particle.fitness = fitness(particle.pos)
            if particle.fitness < particle.best_fitness:
                particle.best_fitness = particle.fitness
                particle.best_pos = np.copy(particle.pos)
            if particle.fitness < swarm_best_fitness:
                swarm_best_fitness = particle.fitness
                swarm_best_pos = np.copy(particle.pos)
                swarm_best_pos_track.append(swarm_best_pos)
    
    print("\nBest solution found:")
    print(swarm_best_pos)
    print(f"fitness of best solution = {swarm_best_fitness:.4e}")

    return swarm_best_pos, swarm_best_pos_track

In [ ]:
def plot_optimization(fitness,minx,maxx,best_pos_track):
    x = np.linspace(minx,maxx,300)
    X1,X2 = np.meshgrid(x,x)
    X = np.dstack([X1,X2])
    Y = np.apply_along_axis(fitness,-1,X)
    plt.figure(figsize=(10,10))
    plt.contourf(X1,X2,Y,levels=25,cmap="Blues")
    x,y = np.array(best_pos_track).T
    plt.plot(x,y,"r")
    plt.plot(*best_pos_track[0],"o",label="start")
    plt.plot(*best_pos_track[-1],"X",label="end")
    plt.legend()
    plt.show()

In [ ]:
np.set_printoptions(precision=6,suppress=True)        

w = 0.729 # inertia
c1 = 1.49445 # cognitive (particle)
c2 = 1.49445 # social (swarm)

dim = 2
num_particles = 50
max_iter = 100
minx, maxx = -10.0, 10.0

### PSO for Rastrigin Function

In [ ]:
def fitness_rastrigin(x):
    x = np.array(x)
    return 10*x.shape[0] + np.sum(x**2 - 10 * np.cos(2*np.pi*x))

best_position,best_pos_track = pso(fitness_rastrigin, max_iter, num_particles, dim,minx ,maxx, w,c1,c2)

In [ ]:
plot_optimization(fitness_rastrigin,-5,5,best_pos_track)

### PSO for Sphere Function

In [ ]:
def fitness_sphere(x):
    x = np.array(x)
    return np.sum(x**2)

best_position, best_pos_track = pso(fitness_sphere, max_iter, num_particles, dim, -10.0, 10.0, w,c1,c2)

In [ ]:
plot_optimization(fitness_sphere,-5,5,best_pos_track)